In [1]:
import json

from flask import Flask, current_app, render_template, request, redirect, url_for
from flask_sqlalchemy import SQLAlchemy

from app import app, db

<img src="static/img/Image_20240323152952.jpg" />

```python
class Issue(db.Model):
    # ...
    # users 属性代表这个关系的面向对象视角
    # 第一个参数表明这个关系的另一端是哪个模型
    # backref 参数向User 模型中添加一个role 属性，从而定义反向关系
    # 这一属性可替代role_id 访问Role 模型，此时获取的是模型对象，而不是外键的值
    # 若User中由两个或以上列定义为Role的外键，则需要提供额外参数以确定所用外键
    users = db.relationship('User', backref='role')
    
class Improvement(db.Model):
    # ...
    role_id = db.Column(db.Integer, db.ForeignKey('roles.id'))
```

In [6]:
# 调用 hello.py 应用中的app实例
with app.app_context() as app_ctx:

    # 手动创建应用上下文，确保正常使用 Flask 应用的全局变量和扩展
    app_ctx.push()
    print(current_app.name)
    # 重置数据库
    db.drop_all()
    db.create_all()
    # 创建 SQLAlchemy 的会话
    session = db.session()

    # 创建Role实例
    admin_role = Role(name='Admin')
    mod_role = Role(name='Moderator')
    user_role = Role(name='User')
    # 将Role实例写入会话
    session.add_all([admin_role, 
                     mod_role, 
                     user_role])
    # 提交会话
    session.commit()

    # 创建User实例
    user_john = User(username='john', role=admin_role, comment='Good', regist=True)
    user_susan = User(username='susan', role=user_role, comment='Ok', regist=True)
    user_david = User(username='david', role=user_role, comment='Bad', regist=True)
    # 将User实例写入会话
    session.add_all([user_john, 
                     user_susan, 
                     user_david])
    # 提交会话
    session.commit()

    # 验证数据
    print("ID of administrator role: %s" % admin_role.id)
    print("ID of moderator role: %s" % mod_role.id)
    print("ID of user role: %s" % user_role.id)
    print("Comment of John: %s" % user_john.comment)
    print("Role of Susan: %s" % user_susan.role)
    print("If David is registed: %s" % user_david.regist)

    # 修改行
    print("Replace name of role {0} with {1}.".format(getattr(admin_role, 'name'), 
                                                              'Administrator'))
    admin_role.name = 'Administrator'
    session.add(admin_role)
    print("New name of administrator role: %s" % admin_role.name)
    # 删除行
    print("Delete {0} role.".format(getattr(mod_role, 'name')))
    session.delete(mod_role)
    # 提交会话
    session.commit()

    # 销毁当前的应用上下文
    app_ctx.pop()

hello
ID of administrator role: 1
ID of moderator role: 2
ID of user role: 3
Comment of John: Good
Role of Susan: <Role 'User'>
If David is registed: True
Replace name of role Admin with Administrator.
New name of administrator role: Administrator
Delete Moderator role.


In [14]:
# 调用 hello.py 应用中的app实例
with app.app_context() as app_ctx:

    # 手动创建应用上下文，确保正常使用 Flask 应用的全局变量和扩展
    app_ctx.push()

    # 直接查询
    print("All roles:")
    print(Role.query.all())
    print("All users:")
    print(User.query.all())

    # 使用过滤器查询
    print("'User' users (filter):")
    print(User.query.filter(User.role.has(name='User')).all())

    # 使用Role类实例进行筛选
    print("'User' users (filter_by):")
    role_of_user = Role.query.filter_by(name='User').first()
    print(User.query.filter_by(role=role_of_user).all())
    # 将query对象转换成SQL查询语句
    print("Translated to SQL query:")
    query = User.query.filter_by(role=role_of_user).statement.compile(compile_kwargs={"literal_binds": True})
    print(str(query))
    

    # 手动移除当前的应用上下文
    app_ctx.pop()

All roles:
[<Role 'Administrator'>, <Role 'User'>]
All users:
[<User 'john'>, <User 'susan'>, <User 'david'>]
'User' users (filter):
[<User 'susan'>, <User 'david'>]
'User' users (filter_by):
[<User 'susan'>, <User 'david'>]
Translated to SQL query:
SELECT users.id, users.username, users.regist, users.comment, users.role_id 
FROM users 
WHERE 3 = users.role_id


如果退出了 shell 会话，这些创建的对象就不会以 Python 对象的形式存在，而
是作为各自数据库表中的行。如果打开了一个新的 shell 会话，就要从数据库中读取行，
再重新创建Python 对象。下面这个例子发起了一个查询，加载名为"User" 的用户角色：

```shell
>>> user_role = Role.query.filter_by(name='User').first()
```

filter_by() 等过滤器在query 对象上调用，返回一个更精确的query 对象。多个过滤器可
以一起调用，直到获得所需结果。
http://docs.sqlalchemy.org

常用的查询过滤器
* filter() 把过滤器添加到原查询上，返回一个新查询
* filter_by() 把等值过滤器添加到原查询上，返回一个新查询
* limit() 使用指定的值限制原查询返回的结果数量，返回一个新查询
* offset() 偏移原查询返回的结果，返回一个新查询
* order_by() 根据指定条件对原查询结果进行排序，返回一个新查询
* group_by() 根据指定条件对原查询结果进行分组，返回一个新查询

常用的查询执行函数
* all() 以列表形式返回查询的所有结果
* first() 返回查询的第一个结果，如果没有结果，则返回None
* first_or_404() 返回查询的第一个结果，如果没有结果，则终止请求，返回404 错误响应
* get() 返回指定主键对应的行，如果没有对应的行，则返回None
* get_or_404() 返回指定主键对应的行，如果没找到指定的主键，则终止请求，返回404 错误响应
* count() 返回查询结果的数量
* paginate() 返回一个Paginate 对象，它包含指定范围内的结果

In [9]:
# 查询行
with app.app_context() as app_ctx:

    app_ctx.push()
    
    # 
    role_of_user = Role.query.filter_by(name='User').first()
    print(role_of_user)
    print("Type of {0}: {1}".format(role_of_user, type(role_of_user)))
    print(role_of_user.users)
    print("Type of 'role_of_user.users': {0}".format(type(role_of_user.users)))
    print("Number of 'role_of_user.users': {0}".format(role_of_user.users.count()))
    users = role_of_user.users.order_by(User.username).all()
    print(users)
    print("Type of {0}: {1}".format(users, type(users)))
    print("Term {0}: {1}".format(0, users[0]))
    print("Role of {0}: {1}".format(users[0], users[0].role))

    app_ctx.pop()

<Role 'User'>
Type of <Role 'User'>: <class 'hello.Role'>
SELECT users.id AS users_id, users.username AS users_username, users.regist AS users_regist, users.comment AS users_comment, users.role_id AS users_role_id 
FROM users 
WHERE ? = users.role_id
Type of 'role_of_user.users': <class 'sqlalchemy.orm.dynamic.AppenderQuery'>
Number of 'role_of_user.users': 2
[<User 'david'>, <User 'susan'>]
Type of [<User 'david'>, <User 'susan'>]: <class 'list'>
Term 0: <User 'david'>
Role of <User 'david'>: <Role 'User'>


In [21]:
# 不同表述方式查询角色为 user_role 的用户
with app.app_context() as app_ctx:

    app_ctx.push()
    
    #result = User.role.has(name='User')
    '''
    EXISTS (SELECT 1 
    FROM roles, users 
    WHERE roles.id = users.role_id AND roles.name = :name_1)
    '''
    #result = User.query.filter(User.role.has(name='User'))
    '''
    SELECT users.id AS users_id, users.username AS users_username, users.regist AS users_regist, users.comment AS users_comment, users.role_id AS users_role_id 
    FROM users 
    WHERE EXISTS (SELECT 1 
    FROM roles 
    WHERE roles.id = users.role_id AND roles.name = ?)
    '''
    #result = User.query.filter(User.role.has(name='User')).all()
    '''
    [<User 'susan'>, <User 'david'>]
    '''



    print(result)

    app_ctx.pop()

[<User 'susan'>, <User 'david'>]
